In [2]:
import os

from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_community.document_loaders import GitLoader

def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

409


In [4]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。
文脈: """
{context}
"""
質問: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
retriever = db.as_retriever()

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | prompt | model | StrOutputParser()

chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティ統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを作成し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクトルストアなど）に対する標準インターフェースを実装し、数百のプロバイダーと統合しています。また、複数のオープンソースライブラリで構成されており、ユーザーは必要に応じてコンポーネントを選択して使用できます。'

In [7]:
from langchain_core.runnables import RunnableParallel

hypothetical_prompt = ChatPromptTemplate.from_template("""\
次の質問に回答する一文を書いてください。
質問: {question}
""")
hypothetical_chain = hypothetical_prompt | model | StrOutputParser()

hyde_rag_chain = RunnableParallel({
    "question": RunnablePassthrough(),
    "context": hypothetical_chain | retriever,
}) | prompt | model | StrOutputParser()
hyde_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、アプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築することができます。\n\n2. **生産化**: LangSmithを使用してアプリケーションを監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、さまざまなプロバイダーと統合し、標準インターフェースを実装しているため、異なるコンポーネントを簡単に切り替えたり、組み合わせたりすることができます。また、LangGraphを使用することで、複雑なアプリケーションのオーケストレーションが可能になり、持続性やストリーミング、ヒューマンインザループなどの機能をサポートします。'

In [8]:
from pydantic import BaseModel, Field

class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(..., description="検索クエリのリスト")

query_generation_prompt = ChatPromptTemplate.from_template("""
質問に対してベクターデータベースから関連文書を検索するために、
3つの異なる検索クエリを生成してください。
距離ベースの類似性検索の限界を克服するために、
ユーザーの質問に対して複数の視点を提供することが目標です。
質問: {question}
""")

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    | (lambda x: x.queries)
)

multi_query_rag_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map(),
} | prompt | model | StrOutputParser()

multi_query_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、アプリケーションのライフサイクルの各段階を簡素化することを目的としています。具体的には、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、チャットモデルや埋め込みモデル、ベクトルストアなどの関連技術に対して標準インターフェースを実装しており、数百のプロバイダーと統合されています。これにより、開発者は異なるコンポーネントを簡単に切り替えたり、組み合わせたりすることができます。\n\nまた、LangChainは、複雑なアプリケーションのオーケストレーションを行うためのLangGraphや、アプリケーションの可視化と評価を行うLangSmithといったツールを提供しています。これにより、開発者はより効率的にアプリケーションを構築し、運用することが可能になります。'

In [12]:
from langchain_core.documents import Document

def reciprocal_rank_fusion(
    retriever_outputs: list[list[Document]],
    k: int = 60,
) -> list[str]:
    # 各ドキュメントのコンテンツ (文字列) とそのスコアの対応を保持する辞書を準備
    content_score_mapping = {}

    # 検索クエリごとにループ
    for docs in retriever_outputs:
        # 検索結果のドキュメントごとにループ
        for rank, doc in enumerate(docs):
            content = doc.page_content
            # 初めて登場したコンテンツの場合はスコアを0で初期化
            if content not in content_score_mapping:
                content_score_mapping[content] = 0
            # (1 / (順位 + k)) のスコアを加算
            content_score_mapping[content] += 1 / (rank + k)

    # スコアの大きい順にソート
    ranked = sorted(content_score_mapping.items(), key=lambda x: x[1], reverse=True)
    return [content for content, _ in ranked]

rag_fusion_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map() | reciprocal_rank_fusion,
} | prompt | model | StrOutputParser()

rag_fusion_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、アプリケーションのライフサイクルの各段階を簡素化することを目的としています。具体的には、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、チャットモデルや埋め込みモデル、ベクトルストアなどの関連技術に対して標準インターフェースを実装し、数百のプロバイダーと統合しています。これにより、開発者は異なるコンポーネントを簡単に組み合わせたり、プロバイダーを切り替えたりすることができます。'

In [9]:
from typing import Any
from langchain_cohere import CohereRerank
from langchain_core.documents import Document

def rerank(inp: dict[str, Any], top_n: int = 3) -> list[Document]:
    question = inp["question"]
    documents = inp["documents"]
    cohere_reranker = CohereRerank(model="rerank-multilingual-v3.0", top_n=top_n)

    return cohere_reranker.compress_documents(documents=documents, query=question)

rerank_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "documents": retriever,
    }
    | RunnablePassthrough.assign(context=rerank)
    | prompt | model | StrOutputParser()
)

rerank_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを作成し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、チャットモデルや埋め込みモデル、ベクトルストアなどの関連技術に対する標準インターフェースを実装しており、数百のプロバイダーと統合されています。また、複数のオープンソースライブラリで構成されており、開発者は必要なコンポーネントを選択して使用することができます。'

In [13]:
from langchain_community.retrievers import TavilySearchAPIRetriever

langchain_document_retriever = retriever.with_config(
    {"run_name": "langchain_document_retriever"}
)

web_retriever = TavilySearchAPIRetriever(k=3).with_config(
    {"run_name": "web_retriever"}
)

In [14]:
from enum import Enum

class Route(str, Enum):
    langchain_document = "langchain_document"
    web = "web"

class RouteOutput(BaseModel):
    route: Route

route_prompt = ChatPromptTemplate.from_template("""\
質問に回答するために適切なRetrieverを選択してください。
質問: {question}
""")

route_chain = (
    route_prompt
    | model.with_structured_output(RouteOutput)
    | (lambda x: x.route)
)

In [15]:
def routed_retriever(inp: dict[str, Any]) -> list[Document]:
    question = inp["question"]
    route = inp["route"]
    if route == Route.langchain_document:
        return langchain_document_retriever.invoke(question)
    elif route == Route.web:
        return web_retriever.invoke(question)
    raise ValueError(f"Unknown retriever: {retriever}")

route_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "route": route_chain,
    }
    | RunnablePassthrough.assign(context=routed_retriever)
    | prompt | model | StrOutputParser()
)

In [ ]:
route_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティ統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクトルストア）に対する標準インターフェースを実装し、数百のプロバイダーと統合しています。また、複数のオープンソースライブラリで構成されており、ユーザーは必要に応じてコンポーネントを選択して使用できます。\n\nさらに、LangChainは、アプリケーションの複雑さに応じて、LangGraphを使用して複雑なオーケストレーションを行うことが推奨されています。LangGraphは、アプリケーションのフローをノードとエッジとしてモデル化し、状態を持つマルチアクターアプリケーションの構築を支援します。'

In [10]:
from langchain_community.retrievers import TavilySearchAPIRetriever

langchain_document_retriever = retriever.with_config(
    {"run_name": "langchain_document_retriever"}
)

web_retriever = TavilySearchAPIRetriever(k=3).with_config(
    {"run_name": "web_retriever"}
)

from enum import Enum

class Route(str, Enum):
    langchain_document = "langchain_document"
    web = "web"

class RouteOutput(BaseModel):
    route: Route

route_prompt = ChatPromptTemplate.from_template("""
質問に回答するために適切なRetrieverを選択してください。
質問: {question}
""")

route_chain = route_prompt | model.with_structured_output(RouteOutput) | (lambda x: x.route)

def routed_retriever(inp: dict[str, Any]) -> list[Document]:
    question = inp["question"]
    route = inp["route"]
    if route == Route.langchain_document:
        return langchain_document_retriever.invoke(question)
    elif route == Route.web:
        return web_retriever.invoke(question)
    raise ValueError(f"Unknown retriever: {retriever}")

route_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "route": route_chain,
    }
    | RunnablePassthrough.assign(context=routed_retriever)
    | prompt | model | StrOutputParser()
)

route_rag_chain.invoke("LangChainの概要を教えて")


'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクトルストアなど）に対する標準インターフェースを実装し、数百のプロバイダーと統合しています。また、複数のオープンソースライブラリで構成されており、ユーザーは必要に応じてこれらのコンポーネントを選択して使用できます。'

In [13]:
from langchain_community.retrievers import BM25Retriever

chroma_retriever = retriever.with_config(
    {"run_name": "chroma_retriever"}
)

bm25_retriever = BM25Retriever.from_documents(documents).with_config(
    {"run_name": "bm25_retriever"}
)

from langchain_core.runnables import RunnableParallel

hybrid_retriever = (
    RunnableParallel({
        "chroma_documents": chroma_retriever,
        "bm25_documents": bm25_retriever,
    })
    | (lambda x: [x["chroma_documents"], x["bm25_documents"]])
    | reciprocal_rank_fusion
)

hybrid_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hybrid_retriever,
    }
    | prompt | model | StrOutputParser()
)

hybrid_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各ステージを簡素化します。具体的には、以下のような機能があります。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティ統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを作成し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクトルストアなど）に対する標準インターフェースを実装し、数百のプロバイダーと統合しています。また、複数のオープンソースライブラリで構成されており、ユーザーは特定のニーズに応じてコンポーネントを選択して使用できます。'